In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from IPython.core.interactiveshell import InteractiveShell
from sklearn.metrics import mean_squared_error
InteractiveShell.ast_node_interactivity = "all" # 한 실행칸에 프린트 여러개 해도 다 출력시키도록 하는 코드.
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# 1번

In [2]:
data = datasets.fetch_california_housing()
y_target = data.target
data = pd.DataFrame(data.data, columns=data.feature_names)

In [3]:
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [4]:
data.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000


In [5]:
data.isna().sum()

MedInc        0
HouseAge      0
AveRooms      0
AveBedrms     0
Population    0
AveOccup      0
Latitude      0
Longitude     0
dtype: int64

## (1) 주어진 데이터셋을 훈련자료 60%, 평가자료 40%로 나누어라

In [6]:
x_train, x_test, y_train, y_test = train_test_split(data, y_target, 
                                                    test_size = 0.4, random_state = 123)

## (2) Ridge regression

In [7]:
_lambda = [0, 1, 10, 30, 50, 100]
for la in _lambda:
    ridge = Ridge(alpha = la)
    mse_scores = cross_val_score(ridge, x_train, y_train, scoring = 'neg_mean_squared_error', cv = 5)
    avg_mse = np.mean(-1 * mse_scores)
    print('alpha = {} -> MSE = {}'.format(la, np.around(avg_mse, decimals = 9)))

alpha = 0 -> MSE = 0.524939318
alpha = 1 -> MSE = 0.52493914
alpha = 10 -> MSE = 0.524956935
alpha = 30 -> MSE = 0.525098812
alpha = 50 -> MSE = 0.525342354
alpha = 100 -> MSE = 0.526199145


alpha는 1이 적절할 것으로 보인다. (이유 : MSE가 가장 작음)

## (3) lambda = 1로 Ridge fitting

Ridge regression train

In [8]:
ridge = Ridge(alpha = 1)
ridge.fit(x_train, y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

가중치 파라미터 추정치는 다음과 같다.

In [9]:
ridge.coef_

array([ 4.42478721e-01,  9.29069265e-03, -1.15089709e-01,  7.35623121e-01,
        9.21138189e-07, -3.53153497e-03, -4.21108801e-01, -4.36009079e-01])

In [10]:
y_preds = ridge.predict(x_test)

test set으로 모델에 대해 구한 R square는 아래와 같다.

In [11]:
r2_score(y_test, y_preds)

0.6090907316775215

# 2번 일반적인 선형 회귀 모형에 비해, Lasso는 ...

정답 : (3) 유연성이 낮고, bias의 증가가 variance의 감소보다 작을 경우 예측 정확도가 향상된다.

왜냐? Lasso는 unbiasedness를 포기하고 variance를 감소시킴으로써 모델의 성능을 향상시키고자 하는 접근법!
유연성이 낮은 것은 "제약"이 가해지기 때문!

# 3번 default.csv 사용

In [12]:
data = pd.read_csv('./default.csv')
data_copy = data.copy()
data.head()

,default,student,balance,income
0,No,No,729.526495,44361.62507
1,No,Yes,817.180407,12106.13470
2,No,No,1073.549164,31767.13895
3,No,No,529.250605,35704.49394
4,No,No,785.655883,38463.49588


In [13]:
data.describe()

,balance,income
count,10000.000000,10000.000000
mean,835.374886,33516.981876
std,483.714985,13336.639563
min,0.000000,771.967729
25%,481.731105,21340.462905
50%,823.636973,34552.644800
75%,1166.308387,43807.729275
max,2654.322576,73554.233500


In [14]:
data.isna().sum()

default    0
student    0
balance    0
income     0
dtype: int64

## (1) dummy 변환 및 표준화

In [15]:
def transform(x):
    y = ['Yes', 'yes', 'y', 'Y']
    n = ['No', 'no', 'n', 'N']
    if x in y:
        return 1
    elif x in n:
        return 0

In [16]:
data['default'] = data['default'].apply(lambda x: transform(x))
data['student'] = data['student'].apply(lambda x: transform(x))

In [17]:
data.dtypes

default      int64
student      int64
balance    float64
income     float64
dtype: object

In [18]:
scaler = StandardScaler()
data['balance'] = scaler.fit_transform(data['balance'].to_numpy().reshape(-1, 1)).reshape(-1)
data['income'] = scaler.fit_transform(data['income'].to_numpy().reshape(-1, 1)).reshape(-1)

In [19]:
data.head()

,default,student,balance,income
0,0,0,-0.218835,0.813187
1,0,1,-0.037616,-1.605496
2,0,0,0.492410,-0.131212
3,0,0,-0.632893,0.164031
4,0,0,-0.102791,0.370915


## (2) logistic regression 훈련

In [20]:
lr_clf = LogisticRegression()
lr_clf.fit(data.iloc[:,1:], data.iloc[:,0])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
lr_clf.coef_
lr_clf.intercept_

array([[-0.65943353,  2.64182192,  0.02127946]])

array([-5.73557497])

## (3) 추정된 파라미터 이용한 이항 로지스틱 회귀모형 식 표현.

- y = -5.73557497 - 0.65943353 * student + 2.64182192 * balance + 0.02127946 * income
- h(x) = 1 / (1 + exp(-y))      >>>>>> y = 1일 확률
- 교수님 프린트 참고한 식임. 

## (4) 예시 default 확률 구하기.

들어온 input이 표준화가 되어있지 않기 때문에 표준화를 해 주어야 한다.

In [22]:
lr_clf.predict_proba(np.array([1, (900 - data_copy['balance'].mean())/data_copy['balance'].std(), 
          (7100 - data_copy['income'].mean())/data_copy['income'].std()]).reshape(1, -1))

array([[0.99772661, 0.00227339]])

즉, 0일 확률이 99.77%이고, 1일 확률이 0.227%이다.

# 4번

In [23]:
np.random.seed(123)
traindt = np.hstack( [np.ones((5, 1)),
                     np.around( np.random.randn(5, 4), 3),
                     np.random.randint(1, 4, (5, 1))])
traindt

array([[ 1.   , -1.086,  0.997,  0.283, -1.506,  2.   ],
       [ 1.   , -0.579,  1.651, -2.427, -0.429,  1.   ],
       [ 1.   ,  1.266, -0.867, -0.679, -0.095,  1.   ],
       [ 1.   ,  1.491, -0.639, -0.444, -0.434,  1.   ],
       [ 1.   ,  2.206,  2.187,  1.004,  0.386,  3.   ]])

In [24]:
softmax_reg = LogisticRegression(multi_class = 'multinomial', penalty = 'l2',
                                 C = 10, random_state = 42, solver = 'newton-cg')
softmax_reg.fit(traindt[:,:-1], traindt[:, -1])

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=42, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

## (1) theta 1

In [25]:
theta_1 = np.array([[5, 2, 3, 1, 4], [2, 4, 3, 1, 2], [3, 4, 1, 5, 4]])
theta_1

array([[5, 2, 3, 1, 4],
       [2, 4, 3, 1, 2],
       [3, 4, 1, 5, 4]])

각 관찰치가 Y의 각 범주에 속할 확률은 다음과 같다. (관찰치는 각 행별로 나타난다. 즉, 1행이 1번째 관찰 set이다)

In [26]:
softmax_reg.predict_proba(theta_1)

array([[5.16272777e-03, 1.54845625e-04, 9.94682427e-01],
       [2.91689618e-03, 4.82263058e-05, 9.97034878e-01],
       [2.54249393e-05, 3.52707007e-05, 9.99939304e-01]])

즉, 각 관찰치는 다음과 같이 class 3으로 분류된다. ( 1 ~ 3번째 관찰치 모두 class 3으로 분류됨.)

In [27]:
softmax_reg.predict(theta_1)

array([3., 3., 3.])

## (2) theta 2 with cross entropy cost function

In [28]:
theta_2 = np.array([[5.5, 2, 3, 1.5, 4], [2, 3.5, 2.5, 1, 1.5], [3, 4, 1, 5, 4]])

In [29]:
softmax_reg.predict_proba(theta_2)
softmax_reg.predict(theta_2)

array([[1.97251763e-03, 1.64314604e-04, 9.97863168e-01],
       [8.49872716e-03, 2.97853100e-04, 9.91203420e-01],
       [2.54249393e-05, 3.52707007e-05, 9.99939304e-01]])

array([3., 3., 3.])

In [34]:
def one_hot(y, p):
    res = np.zeros((len(p), len(p)))
    for i in range(len(p)):
        res[i][len(p)-1] = 1
    return res

In [35]:
# 크로스 엔트로피 비용함수
def cross_entropy_loss(y, p):
    return (one_hot(y, p) * (-np.log(p))).sum() / len(p)

In [36]:
p1 = softmax_reg.predict_proba(theta_1)
y1 = softmax_reg.predict(theta_1)
p2 = softmax_reg.predict_proba(theta_2)
y2 = softmax_reg.predict(theta_2)

In [37]:
cross_entropy_loss(y1, p1)

cross_entropy_loss(y2, p2)

0.0027873288859272282

0.0036784381930716667

theta_1의 경우가 크로스 엔트로피 비용함수 결과가 낮기 때문에 더 적절한 파라미터 행렬이라고 볼 수 있다.